# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果
# [作業重點]
仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5])
仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7])
試著回答上述執行結果的觀察
作業1
參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，
與其他數值型欄位一起預估生存機率

In [34]:
# 做完特徵工程前的所有準備 (與前範例相同)
import os
import numpy as np
import pandas as pd
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

dir_path = '../data/'
f_app_train = os.path.join(dir_path, "titanic_train.csv")
df_train = pd.read_csv(f_app_train)
f_app_test = os.path.join(dir_path, "titanic_test.csv")
df_test = pd.read_csv(f_app_test)

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])

df.head()

Pclass                                               Name     Sex   Age  \
0       3                            Braund, Mr. Owen Harris    male  22.0   
1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2       3                             Heikkinen, Miss. Laina  female  26.0   
3       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4       3                           Allen, Mr. William Henry    male  35.0   

   SibSp  Parch            Ticket     Fare Cabin Embarked  
0      1      0         A/5 21171   7.2500   NaN        S  
1      1      0          PC 17599  71.2833   C85        C  
2      0      0  STON/O2. 3101282   7.9250   NaN        S  
3      1      0            113803  53.1000  C123        S  
4      0      0            373450   8.0500   NaN        S

In [35]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
    
print('Number of Object Features : {}'.format(len(object_features)))
print('Object Features : {}'.format(object_features))


# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

Number of Object Features : 5
Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


Name     Sex  \
0                            Braund, Mr. Owen Harris    male   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   
2                             Heikkinen, Miss. Laina  female   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   
4                           Allen, Mr. William Henry    male   

             Ticket Cabin Embarked  
0         A/5 21171  None        S  
1          PC 17599   C85        C  
2  STON/O2. 3101282  None        S  
3            113803  C123        S  
4            373450  None        S

In [36]:
# 觀察欄位相異值數量
df.select_dtypes(include=["object"]).apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [37]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.780004837244799


Name  Sex  Ticket  Cabin  Embarked
0   155    1     720    185         3
1   286    0     816    106         0
2   523    0     914    185         3
3   422    0      65     70         3
4    22    1     649    185         3

In [38]:
# 加上 'Cabin' 欄位的計數編碼
# 第一行 : df.groupby(['Cabin']) 會輸出 df 以 'Cabin' 群聚後的結果, 但因為群聚一類只會有一個值, 因此必須要定義運算
# 例如 df.groupby(['Cabin']).size(), 但欄位名稱會變成 size, 要取別名就需要用語法 df.groupby(['Cabin']).agg({'Cabin_Count':'size'})
# 這樣出來的計數欄位名稱會叫做 'Cabin_Count', 因為這樣群聚起來的 'Cabin' 是 index, 所以需要 reset_index() 轉成一欄
# 因此第一行的欄位, 在第三行按照 'Cabin_Count' 排序後, 最後的 DataFrame 輸出如 Out[5]
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
# 但是上面資料表結果只是 'Ticket' 名稱對應的次數, 要做計數編碼還需要第二行 : 將上表結果與原表格 merge, 合併於 'Ticket' 欄位
# 使用 how='left' 是完全保留原資料表的所有 index 與順序
df = pd.merge(df, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


Cabin  Cabin_Count
185             None         1014
80       C23 C25 C27            6
184               G6            5
47   B57 B59 B63 B66            5
60           B96 B98            4
180               F2            4
117                D            4
79           C22 C26            4
181              F33            4
183               F4            4

In [39]:
# 印出來看看, 加了計數編碼的資料表 df 有何不同
df.head()


Name     Sex  \
0                            Braund, Mr. Owen Harris    male   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   
2                             Heikkinen, Miss. Laina  female   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   
4                           Allen, Mr. William Henry    male   

             Ticket Cabin Embarked  Cabin_Count  
0         A/5 21171  None        S         1014  
1          PC 17599   C85        C            2  
2  STON/O2. 3101282  None        S         1014  
3            113803  C123        S            2  
4            373450  None        S         1014

In [40]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Count'] = df['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7856230275549181


Name  Sex  Ticket  Cabin  Embarked  Cabin_Count
0   155    1     720    185         3         1014
1   286    0     816    106         0            2
2   523    0     914    185         3         1014
3   422    0      65     70         3            2
4    22    1     649    185         3         1014

In [41]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
# 這邊的雜湊編碼, 是直接將 'Cabin' 的名稱放入雜湊函數的輸出數值, 為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 這邊的 10 是隨機選擇, 不一定要用 10, 同學可以自由選擇購小的數字試看看. 基本上效果都不會太好
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7900795343761782


Name  Sex  Ticket  Cabin  Embarked  Cabin_Hash
0   155    1     720    185         3           0
1   286    0     816    106         0           3
2   523    0     914    185         3           0
3   422    0      65     70         3           5
4    22    1     649    185         3           0

In [48]:
# 'Cabin'均值編碼 (Mean Encoder) + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, "mean_{}".format(c)]
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived',"mean_Name", "mean_Sex", "mean_Ticket", "mean_Embarked", "mean_Cabin_Count"] , axis=1)

data.head()

mean_Cabin
0    0.299854
1    1.000000
2    0.299854
3    0.500000
4    0.299854

In [49]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 'Cabin'均值編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
    
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
df_temp['Cabin_Count'] = df['Cabin_Count']
df_temp['Cabin_mean'] = data['mean_Cabin']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.826079310245774


Name  Sex  Ticket  Cabin  Embarked  Cabin_Hash  Cabin_Count  Cabin_mean
0   155    1     720    185         3           0         1014    0.299854
1   286    0     816    106         0           3            2    1.000000
2   523    0     914    185         3           0         1014    0.299854
3   422    0      65     70         3           5            2    0.500000
4    22    1     649    185         3           0         1014    0.299854